<a href="https://colab.research.google.com/github/allenwang0713/tebaml0315/blob/main/imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84125825/84125825 [==============================] - 2s 0us/step


In [2]:

import os
import glob
import pandas as pd
def getdata(mid):
    dn = os.path.dirname(dataset)
    posfn = glob.glob(os.path.join(dn, "aclImdb", mid, "pos", "*"))
    negfn = glob.glob(os.path.join(dn, "aclImdb", mid, "neg", "*"))
    contents = []
    for fn in posfn + negfn:
        with open(fn, encoding="utf-8") as f:
            contents.append(f.read())
    df = pd.DataFrame({
        "content":contents,
        "sentiment":[1] * len(posfn) + [0] * len(negfn)
    })
    return df
train_df = getdata("train")
test_df = getdata("test")

In [3]:
test_df

,content,sentiment
0,ROCK STAR / (2001) *** (out of four)<br /><br ...,1
1,The Cure is a fantastic film about a boy with ...,1
2,"Not having seen this film in quite some time, ...",1
3,One of the flat-out drollest movies of all-tim...,1
4,"NVA combines eastalgia-humor, military comedy ...",1
...,...,...
24995,Strangers with candy overacts in all the wrong...,0
24996,"What a disaster! Normally, when one critiques ...",0
24997,The Robot vs. the Aztec Mummy was one of the s...,0
24998,I'm not prone to ranting and my expectations w...,0


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
# 3000種常用詞彙+1padding(0): 美一篇文章進入的時候只取512在常用詞彙列表的詞, 每一個詞化做100維度的語意像量
layers = [
    # 沒有激活, 3001(種詞彙) * 100 -> 300100
    Embedding(input_dim=3001, output_dim=100, mask_zero=True, input_length=512),
    GlobalAveragePooling1D(),
    Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 512, 100)          300100    
                                                                 
 global_average_pooling1d (G  (None, 100)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 2)                 202       
                                                                 
Total params: 300,302
Trainable params: 300,302
Non-trainable params: 0
_________________________________________________________________
